**SOW-MKI49: Neural Information Processing Systems**  
*Weeks 8 and 9: Assignment (100 points + 10 bonus points)*  
Adapted from Andrew Ng's deep learning course.

# Deep Learning & Art: Neural Style Transfer

In this assignment, you will learn about Neural Style Transfer. This algorithm was created by Gatys et al. (2015) (https://arxiv.org/abs/1508.06576). 

**In this assignment, you will:**
- Implement the neural style transfer algorithm 
- Generate novel artistic images using your algorithm 

Most of the algorithms you've studied optimize a cost function to get a set of parameter values. In Neural Style Transfer, you'll optimize a cost function to get pixel values!

## 1 - Problem Statement

Neural Style Transfer (NST) is one of the most fun techniques in deep learning. It merges two images, namely, a "content" image (C) and a "style" image (S), to create a "generated" image (G). The generated image G combines the "content" of the image C with the "style" of image S. 

In this example, you are going to generate an image of the Heyendaal castle in Nijmegen (content image C), mixed with a painting by Vincent van Gogh, a leader of the post-impressionist movement (style image S).

Let's see how you can do this. 

In [14]:
import os
import sys
import scipy.io
import scipy.misc
import tensorflow as tf
import numpy as np
import wget
import imageio
import skimage
import matplotlib.pyplot as plt
#import tensornets as tn
from matplotlib.pyplot import imshow
from PIL import Image
from keras.applications import vgg19
from keras import backend as k
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg19 import preprocess_input
from skimage.transform import resize
from keras import backend as K
from scipy.optimize import fmin_l_bfgs_b

%matplotlib inline

## 2 - Transfer Learning

Neural Style Transfer (NST) uses a previously trained convolutional network, and builds on top of that. The idea of using a network trained on a different task and applying it to a new task is called transfer learning. 

Following the original NST paper (https://arxiv.org/abs/1508.06576), we will use the VGG network. Specifically, we'll use VGG-19, a 19-layer version of the VGG network. This model has already been trained on the very large ImageNet database, and thus has learned to recognize a variety of low level features (at the earlier layers) and high level features (at the deeper layers). 

Run the following code to load parameters from the VGG model. This may take a few seconds. 

## 3 - Neural Style Transfer 

We will build the NST algorithm in three steps:

- Build the content cost function $J_{content}(C,G)$
- Build the style cost function $J_{style}(S,G)$
- Put it together to get $J(G) = \alpha J_{content}(C,G) + \beta J_{style}(S,G)$. 

### 3.1 - Computing the content cost

In our running example, the content image C will be the picture of the Heyendaal castle in Nijmegen. Run the code below to see a picture of the Heyendaal castle.

In [ ]:
url = "https://upload.wikimedia.org/wikipedia/commons/thumb/9/9e/Heyendaal.jpg/640px-Heyendaal.jpg"
filename = wget.download(url)
content_image = imageio.imread("640px-Heyendaal.jpg")
imshow(content_image)

** 3.1.1 - How do you ensure the generated image G matches the content of the image C?**

As we saw in lecture, the earlier (shallower) layers of a ConvNet tend to detect lower-level features such as edges and simple textures, and the later (deeper) layers tend to detect higher-level features such as more complex textures as well as object classes. 

We would like the "generated" image G to have similar content as the input image C. Suppose you have chosen some layer's activations to represent the content of an image. In practice, you'll get the most visually pleasing results if you choose a layer in the middle of the network--neither too shallow nor too deep. (After you have finished this exercise, feel free to come back and experiment with using different layers, to see how the results vary.)

So, suppose you have picked one particular hidden layer to use. Now, set the image C as the input to the pretrained VGG network, and run forward propagation. Let $a^{(C)}$ be the hidden layer activations in the layer you had chosen. This will be a $n_H \times n_W \times n_C$ tensor. Repeat this process with the image G: Set G as the input, and run forward progation. Let $a^{(G)}$ be the corresponding hidden layer activation. We will define as the content cost function as:

$$J_{content}(C,G) =  \frac{1}{4 \times n_H \times n_W \times n_C}\sum _{ \text{all entries}} (a^{(C)} - a^{(G)})^2\tag{1} $$

Here, $n_H, n_W$ and $n_C$ are the height, width and number of channels of the hidden layer you have chosen, and appear in a normalization term in the cost. For clarity, note that $a^{(C)}$ and $a^{(G)}$ are the volumes corresponding to a hidden layer's activations. In order to compute the cost $J_{content}(C,G)$, it might also be convenient to unroll these 3D volumes into a 2D matrix, as shown below. (Technically this unrolling step isn't needed to compute $J_{content}$, but it will be good practice for when you do need to carry out a similar operation later for computing the style const $J_{style}$.)

<img src="https://i.postimg.cc/YqbBM5jM/NST-LOSS.png">



**Exercise (10 points)**

Compute the "content cost" using Chainer. 

**Instructions**: The 3 steps to implement this function are:
1. Retrieve dimensions from a_G.
2. Unroll a_C and a_G.
3. Compute the content cost.

In [2]:
def compute_content_cost(a_C, a_G):
    """
    Computes the content cost
    
    Arguments:
    a_C -- tensor of dimension (1, n_C, n_H, n_W), hidden layer activations representing content of the image C 
    a_G -- tensor of dimension (1, n_C, n_H, n_W), hidden layer activations representing content of the image G
    
    Returns: 
    J_content -- scalar that you compute using equation 1 above.
    """
    
    ### START CODE HERE ###
    n_C = K.shape(a_G)[3]
    n_H = K.shape(a_G)[1]
    n_W = K.shape(a_G)[2]
    
    J_1 = (1 / (4 * n_H * n_W * n_C))
    J_2 =  k.sum(k.pow((a_C - a_G), 2))
    J_2 = tf.keras.backend.cast(J_2, 'float64')
    ### END CODE HERE ###
    
    return J_1 * J_2

### 3.2 - Computing the style cost

For our running example, we will use the following style image: 

In [ ]:
url = "https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/606px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg"
wget.download(url)
style_image = imageio.imread("606px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg")
imshow(style_image)

### 3.2.1 - Style matrix

The style matrix is also called a "Gram matrix." In linear algebra, the Gram matrix G of a set of vectors $(v_{1},\dots ,v_{n})$ is the matrix of dot products, whose entries are ${\displaystyle G_{ij} = v_{i}^T v_{j} = np.dot(v_{i}, v_{j})  }$. In other words, $G_{ij}$ compares how similar $v_i$ is to $v_j$: If they are highly similar, you would expect them to have a large dot product, and thus for $G_{ij}$ to be large. 

Note that there is an unfortunate collision in the variable names used here. We are following common terminology used in the literature, but $G$ is used to denote the Style matrix (or Gram matrix) as well as to denote the generated image $G$. We will try to make sure which $G$ we are referring to is always clear from the context. 

In NST, you can compute the Style matrix by multiplying the "unrolled" filter matrix with their transpose:

<img src="https://i.postimg.cc/Fzx5fqZb/NST-GM.png">

The result is a matrix of dimension $(n_C,n_C)$ where $n_C$ is the number of filters. The value $G_{ij}$ measures how similar the activations of filter $i$ are to the activations of filter $j$. 

One important part of the gram matrix is that the diagonal elements such as $G_{ii}$ also measures how active filter $i$ is. For example, suppose filter $i$ is detecting vertical textures in the image. Then $G_{ii}$ measures how common  vertical textures are in the image as a whole: If $G_{ii}$ is large, this means that the image has a lot of vertical texture. 

By capturing the prevalence of different types of features ($G_{ii}$), as well as how much different features occur together ($G_{ij}$), the Style matrix $G$ measures the style of an image. 



**Exercise (10 points)**:
Using Chainer, implement a function that computes the Gram matrix of a matrix A. The formula is: The gram matrix of A is $G_A = AA^T$.

In [3]:
def gram_matrix(A):
    """
    Argument:
    A -- matrix of shape (n_C, n_H*n_W)
    
    Returns:
    GA -- Gram matrix of A, of shape (n_C, n_C)
    """
    
    ### START CODE HERE ###
    GA = K.dot(A, K.transpose(A))
    ### END CODE HERE ###
    
    return GA

### 3.2.2 - Style cost

After generating the Style matrix (Gram matrix), your goal will be to minimize the distance between the Gram matrix of the "style" image S and that of the "generated" image G. For now, we are using only a single hidden layer $a^{[l]}$, and the corresponding style cost for this layer is defined as: 

$$J_{style}^{[l]}(S,G) = \frac{1}{4 \times {n_C}^2 \times (n_H \times n_W)^2} \sum _{i=1}^{n_C}\sum_{j=1}^{n_C}(G^{(S)}_{ij} - G^{(G)}_{ij})^2\tag{2} $$

where $G^{(S)}$ and $G^{(G)}$ are respectively the Gram matrices of the "style" image and the "generated" image, computed using the hidden layer activations for a particular hidden layer in the network.  


**Exercise (10 points)**:

Compute the style cost for a single layer. 

The 3 steps to implement this function are:
1. Retrieve dimensions from the hidden layer activations a_G
2. Unroll the hidden layer activations a_S and a_G into 2D matrices, as explained in the picture above.
3. Compute the Style matrix of the images S and G. (Use the function you had previously written.) 
4. Compute the Style cost.

In [4]:
def compute_layer_style_cost(a_S, a_G):
    """
    Arguments:
    a_S -- tensor of dimension (1, n_C, n_H, n_W), hidden layer activations representing style of the image S 
    a_G -- tensor of dimension (1, n_C, n_H, n_W), hidden layer activations representing style of the image G
    
    Returns: 
    J_style_layer -- tensor representing a scalar value, style cost defined above by equation (2)
    """
    
    ### START CODE HERE ###
    n_C = K.shape(a_G)[1] # Has to be changed (because of chainer vs Keras)
    n_H = K.shape(a_G)[2] # Has to be changed (because of chainer vs Keras)
    n_W = K.shape(a_G)[3] # Has to be changed (because of chainer vs Keras)
    
    a_S_unrolled = K.reshape(a_S, (n_C, n_H * n_W))
    a_G_unrolled = K.reshape(a_G, (n_C, n_H * n_W))
    
    gram_S = gram_matrix(a_S_unrolled)
    gram_G = gram_matrix(a_G_unrolled)
    
    difference_grams_squared = K.pow((gram_S - gram_G), 2)
    J_1 = (1/(4 * (n_C ** 2) * (n_H * n_W) ** 2))
    J_2 = K.sum(difference_grams_squared)
    J_2 = tf.keras.backend.cast(J_2, 'float64')
    ### END CODE HERE ###
    
    return J_1 * J_2

### 3.2.3 Style Weights

So far you have captured the style from only one layer. We'll get better results if we "merge" style costs from several different layers. After completing this exercise, feel free to come back and experiment with different weights to see how it changes the generated image $G$. But for now, this is a pretty reasonable default: 

In [5]:
style_layers = [('block1_conv1', 0.2), ('block2_conv1', 0.2), ('block3_conv1', 0.2), ('block4_conv1', 0.2), ('block5_conv1', 0.2)]

You can combine the style costs for different layers as follows:

$$J_{style}(S,G) = \sum_{l} \lambda^{[l]} J^{[l]}_{style}(S,G)$$

where the values for $\lambda^{[l]}$ are given in `STYLE_LAYERS`. 


**Exercise (10 points)**

Implement the `compute_style_cost(...)` function. It should simply call the `compute_layer_style_cost(...)` several times, and weight their results using the values in `STYLE_LAYERS`.

In [6]:
def compute_style_cost(STYLE_LAYERS):
    """
    Computes the overall style cost from several chosen layers
    
    Arguments:
    model -- our tensorflow model
    STYLE_LAYERS -- A python list containing:
                        - the names of the layers we would like to extract style from
                        - a coefficient for each of them
    
    Returns: 
    J_style -- tensor representing a scalar value, style cost defined above by equation (2)
    """
    
    ### START CODE HERE ###
    J_style =0 
 
    for (layer, l_ambda) in STYLE_LAYERS:
        
        activation_s = sModel.get_layer(layer).output    
        activation_g = gModel.get_layer(layer).output   
        layer_cost = compute_layer_style_cost(activation_s, activation_g)
        J_style += l_ambda * layer_cost
    ### END CODE HERE ###

    return J_style

Finally, let's create a cost function that minimizes both the style and the content cost. The formula is: 

$$J(G) = \alpha J_{content}(C,G) + \beta J_{style}(S,G)$$

**Exercise (10 points)**

Implement the total cost function which includes both the content cost and the style cost.

In [7]:
def total_cost(J_content, J_style, alpha = 10, beta = 40):
    """
    Computes the total cost function
    
    Arguments:
    J_content -- content cost coded above
    J_style -- style cost coded above
    alpha -- hyperparameter weighting the importance of the content cost
    beta -- hyperparameter weighting the importance of the style cost
    
    Returns:
    J -- total cost as defined by the formula above.
    """
    
    ### START CODE HERE ###
    J = alpha * J_content + beta * J_style
    ### END CODE HERE ###
    
    return J

In [27]:
def get_total_cost(gImPlaceholder):
    a_C = cModel.get_layer(cLayerName).output    
    a_G = gModel.get_layer(cLayerName).output
    J_content = compute_content_cost(a_C, a_G)
    J_style = compute_style_cost(style_layers)
    J_total = total_cost(J_content, J_style)
    return J_total

## 4 - Solving the optimization problem (50 points)

Finally, let's put everything together to implement Neural Style Transfer!


Here's what the program will have to do:
<font color='purple'>

1. Load the content image 
2. Load the style image
3. Randomly initialize the image to be generated 
4. Load the VGG16 model
5. Build a Chainer Model:
    6. Run the content image through the VGG16 model and compute the content cost
    7. Run the style image through the VGG16 model and compute the style cost
    8. Compute the total cost
    9. Define the optimizer and the learning rate
10. Initialize the Chainer model and run it for a large number of iterations, updating the generated image at every step.

Let's load, reshape, and normalize our "content" image (the Louvre museum picture):

In [8]:
# resize content and style image so that they have the same dimensions
# also add extra dimension, to account for batch size
targetWidth = 447
targetHeight = 606
style_image = imageio.imread("606px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg")
style_image = resize(style_image, (targetWidth, targetHeight))
style_image = K.variable(preprocess_input(np.expand_dims(style_image, axis=0)), dtype='float32')

content_image = imageio.imread("640px-Heyendaal.jpg")
content_image = resize(content_image, (targetWidth, targetHeight))
content_image = K.variable(preprocess_input(np.expand_dims(content_image, axis=0)), dtype='float32')

### possibly normalize images as well but maybe not necessary

generated_image =  np.random.rand(targetWidth, targetHeight, 3).astype('float64')
generated_image = preprocess_input(np.expand_dims(generated_image, axis =0))
generated_image_placeholder = tf.get_variable('generated_img', shape=(1, targetWidth, targetHeight, 3), dtype = 'float32')
#generated_image = K.variable(preprocess_input(np.expand_dims(generated_image, axis=0)), dtype='float32')

C:\Users\Djamari\Anaconda2\envs\py3k6\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [9]:
generated_image_ = tf.convert_to_tensor(generated_image, dtype = 'float32')

In [ ]:
generated_image_placeholder.dtype

In [ ]:
(generated_image_.dtype)

In [28]:
def get_gradient(gImArr):
    #TODO: change this function to something less copy-pasta
    #pdb.set_trace()
    if gImArr.shape != (1, targetWidth, targetHeight, 3):
        gImArr = gImArr.reshape((1, targetWidth, targetHeight, 3))
    grad_fcn = K.function([gModel.input], K.gradients(compute_loss(gModel.input), [gModel.input]))
    grad = grad_fcn([gImArr])[0].flatten().astype('float64')
    return grad

In [29]:
def compute_loss(generatedImage):
    if gImArr.shape != (1, targetWidth, targetWidth, 3):
        gImArr = gImArr.reshape((1, targetWidth, targetHeight, 3))
    loss_fcn = K.function([gModel.input], [get_total_loss(gModel.input)])
    return loss_fcn([gImArr])[0].astype('float64')


In [30]:
'''clogits = tn.VGG19(content_image)
cModel = tf.identity(clogits, name='clogits')
slogits = tn.VGG19(style_image)
sModel = tf.identity(slogits, name='slogits')
glogits = tn.VGG19(content_image)
gModel = tf.identity(generated_image_placeholder, name='glogits')'''

#tf.reset_default_graph()
sess = tf.InteractiveSession()

cModel = vgg19.VGG19(include_top=False, weights='imagenet', input_tensor=content_image)
sModel = vgg19.VGG19(include_top=False, weights='imagenet', input_tensor=style_image)
gModel = vgg19.VGG19(include_top=False, weights='imagenet', input_tensor=generated_image_placeholder)

vars_train = gModel.trainable_weights

'''cModel.trainable = False
sModel.trainable = False
gModel.trainable = False'''

cLayerName = 'block4_conv2'
sLayerNames = [
                'block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                ]

iterations = 5


'''a_S = sModel.get_layer(layer).output    
a_G = gModel.get_layer(cLayerName).output 
a_C = cModel.get_layer(cLayerName).output

J_content = compute_content_cost(a_C, a_G)

J_style = compute_style_cost(sLayerNames)'''

'''#gradients = K.gradients(compute_loss(), gModel.input)

J_total = compute_loss()
#sess.run(cModel.predict(content_image))
#sess.run(sModel.predict(style_image))
optimizer = tf.train.AdamOptimizer(learning_rate = 1e-4).minimize(J_total)
#generated_image = generated_image.flatten()
'''
'''def model_nn(sess, input_image, num_iterations = iterations):
    
    sess.run(tf.global_variables_initializer())
    sess.run(tf.assign(generated_image_placeholder, input_image))
    
    for i in range(num_iterations):
        print('Iteration number: ', i)
        # Run the session on the optimizer to minimize the total cost
        sess.run(optimizer)
        # Compute the generated image by running the session on the current model['input']
        generated_image1 = sess.run(gModel)
        
    return generated_image1'''

# TODO: change the fmin_l_bfgs_b function for less copy-pasta
xopt, f_val, info= fmin_l_bfgs_b(compute_loss, generated_image, fprime=get_gradient, maxiter=iterations)

'''
P = get_feature_reps(x=cImArr, layer_names=[cLayerName], model=cModel)[0]
As = get_feature_reps(x=sImArr, layer_names=sLayerNames, model=sModel)
ws = np.ones(len(sLayerNames))/float(len(sLayerNames))

iterations = 600
x_val = gIm0.flatten()
xopt, f_val, info= fmin_l_bfgs_b(calculate_loss, x_val, fprime=get_grad,
maxiter=iterations, disp=True)
'''

ResourceExhaustedError: OOM when allocating tensor with shape[3,3,512,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: block4_conv2_19/random_uniform/RandomUniform = RandomUniform[T=DT_INT32, dtype=DT_FLOAT, seed=87654321, seed2=3848874, _device="/job:localhost/replica:0/task:0/device:GPU:0"](block4_conv2_19/random_uniform/shape)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'block4_conv2_19/random_uniform/RandomUniform', defined at:
  File "C:\Users\Djamari\Anaconda2\envs\py3k6\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Djamari\Anaconda2\envs\py3k6\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Djamari\Anaconda2\envs\py3k6\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users

In [ ]:
xOut = postprocess_array(xopt)

In [ ]:
def postprocess_array(x):
    # Zero-center by mean pixel
    if x.shape != (targetWidth, targetHeight, 3):
        x = x.reshape((targetWidth, targetHeight, 3))
    x[..., 0] += 103.939
    x[..., 1] += 116.779
    x[..., 2] += 123.68
    # 'BGR'->'RGB'
    x = x[..., ::-1]
    x = np.clip(x, 0, 255)
    x = x.astype('uint8')
    return x

## 5 - Test with your own image (10 bonus points)

Finally, you can also rerun the algorithm on your own images! 

You can also tune your hyperparameters: 
- Which layers are responsible for representing the style? STYLE_LAYERS
- How many iterations do you want to run the algorithm? num_iterations
- What is the relative weighting between content and style? alpha/beta

## 7 - Conclusion

Great job on completing this assignment! You are now able to use Neural Style Transfer to generate artistic images. This is also your first time building a model in which the optimization algorithm updates the pixel values rather than the neural network's parameters. Deep learning has many different types of models and this is only one of them! 

<font color='blue'>
What you should remember:
- Neural Style Transfer is an algorithm that given a content image C and a style image S can generate an artistic image
- It uses representations (hidden layer activations) based on a pretrained ConvNet. 
- The content cost function is computed using one hidden layer's activations.
- The style cost function for one layer is computed using the Gram matrix of that layer's activations. The overall style cost function is obtained using several hidden layers.
- Optimizing the total cost function results in synthesizing new images.

### References:

The Neural Style Transfer algorithm was due to Gatys et al. (2015). Harish Narayanan and Github user "log0" also have highly readable write-ups from which we drew inspiration. The pre-trained network used in this implementation is a VGG network, which is due to Simonyan and Zisserman (2015).

- Leon A. Gatys, Alexander S. Ecker, Matthias Bethge, (2015). A Neural Algorithm of Artistic Style (https://arxiv.org/abs/1508.06576) 
- Harish Narayanan, Convolutional neural networks for artistic style transfer. https://harishnarayanan.org/writing/artistic-style-transfer/
- Log0, TensorFlow Implementation of "A Neural Algorithm of Artistic Style". http://www.chioka.in/tensorflow-implementation-neural-algorithm-of-artistic-style
- Karen Simonyan and Andrew Zisserman (2015). Very deep convolutional networks for large-scale image recognition (https://arxiv.org/pdf/1409.1556.pdf)